# Clone Github

In [1]:
import os
#print(os.getcwd())
os.chdir('../content')
#os.chdir('../')
!ls

datalab  syllable-aware


In [2]:
!rm -R syllable-aware/
!ls

datalab


In [3]:
!git clone https://github.com/nlpchile/syllable-aware.git
!ls

Cloning into 'syllable-aware'...
remote: Counting objects: 523, done.
remote: Compressing objects: 100% (137/137), done.
remote: Total 523 (delta 77), reused 40 (delta 11), pack-reused 375
Receiving objects: 100% (523/523), 4.66 MiB | 33.62 MiB/s, done.
Resolving deltas: 100% (312/312), done.
datalab  syllable-aware


In [4]:
import os
os.chdir('syllable-aware')
!ls

calculate_Lmax.py     LSTM_CLASS.ipynb		   process_text.py
callback_losswise.py  lstmClass.py		   README.md
data		      LSTM.py			   separadorSilabas.py
generate_percent.py   on_epoch_end AND sample.txt  test_perplexity.py
generators.py	      perplexity.py		   test_tokens_selectors.py
kmp.py		      process_corpus.py		   token_selectors.py


# Silabas.py + # coding=utf-8

In [5]:
%%writefile Silabas.py

# coding=utf-8

def silabas(word, sep='-'):
    '''Devuelve la palabra introducida separada en silabas
    separadas por el caracter 'sep', por defecto '-'.
    No introducir signos de puntuacion.'''
    word = word.lower()
    l = ['r','l']
    o = ['p','b','f','t','d','c','k','g']
    c = ['b','c','ch','d','f','g','h','j','k','l','ll','m','n','ñ','p','q','r','rr','s','t','v','x','y','z']
    a = ['a','e','o','á','é','ó','í','ú']
    i = ['i','u','ü']
    letras = []
    estructura = ''
    j = 0

    # fix
    if word in c and word != 'y':
        raise TypeError("Estructura de sílaba incorrecta en la palabra {0}".format(word))

    while j < len(word):
        if j == 0:
            if word[j] == 'p' and word[j+1] == 's':
                letras.append('ps')
                estructura += 'C'
                j += 2
                continue
            elif word[j] == 'p' and word[j+1] == 'n':
                letras.append('pn')
                estructura += 'C'
                j += 2
                continue
            elif word[j] == 'p' and word[j+1] == 't':
                letras.append('pt')
                estructura += 'C'
                j += 2
                continue
            elif word[j] == 'g' and word[j+1] == 'n':
                letras.append('gn')
                estructura += 'C'
                j += 2
                continue
        if j < len(word) - 1:
            if word[j] == 'c' and word[j+1] == 'h':
                letras.append('ch')
                estructura += 'C'
                j += 2
                continue
            elif word[j] == 'l' and word[j+1] == 'l':
                letras.append('ll')
                estructura += 'C'
                j += 2
                continue
            elif word[j] == 'r' and word[j+1] == 'r':
                letras.append('rr')
                estructura += 'C'
                j += 2
                continue
        if word[j] in a:
            letras.append(word[j])
            estructura += 'A'
            j += 1
            continue
        elif word[j] in i:
            letras.append(word[j])
            estructura += 'I'
            j += 1
            continue
        elif word[j] in l:
            letras.append(word[j])
            estructura += 'L'
            j += 1
            continue
        elif word[j] in o:
            letras.append(word[j])
            estructura += 'O'
            j += 1
            continue
        elif word[j] in c:
            letras.append(word[j])
            estructura += 'C'
            j += 1
            continue
        else:
            raise TypeError("No se reconoce el carácter '{0}' como una letra del castellano.".format(word[j]))
    estructura += 'C'
    letras.append('')
    salida = []
    j = 0
    silaba = ''
    while j < len(letras):
        if letras[j] == '':
            break
        silaba += letras[j]
        if estructura[j] == 'A':
            if estructura[j+1] == 'A':
                salida.append(silaba)
                silaba = ''
                j+=1
                continue
            elif estructura[j+1] == 'I':
                j+=1
                continue
            elif estructura[j+1] == 'O':
                if estructura[j+2] in 'AIL':
                    if letras[j+1] == 'd' and letras[j+2] == 'l':
                        salida.append(silaba + letras[j+1])
                        silaba = ''
                        j += 2
                        continue
                    salida.append(silaba)
                    silaba = ''
                    j+=1
                    continue
                else:
                    if letras[j+2] == 's' and estructura[j+3] in 'LCO':
                        salida.append(silaba + letras[j+1] + letras[j+2])
                        silaba = ''
                        j += 3
                        continue
                    salida.append(silaba + letras[j+1])
                    silaba = ''
                    j += 2
                    continue
            else:
                if j+2 < len(letras):
                    if estructura[j+2] in 'AI':
                        salida.append(silaba)
                        silaba = ''
                        j+=1
                        continue
                    else:
                        if letras[j+2] == 's' and estructura[j+3] in 'LCO':
                            salida.append(silaba + letras[j+1] + letras[j+2])
                            silaba = ''
                            j += 3
                            continue
                        salida.append(silaba + letras[j+1])
                        silaba = ''
                        j += 2
                        continue
                else:
                    salida.append(silaba + letras[j+1])
                    silaba = ''
                    j += 2
                    continue
        elif estructura[j] == 'I':
            if estructura[j+1] in 'AI':
                j+=1
                continue
            elif estructura[j+1] == 'O':
                if estructura[j+2] in 'AIL':
                    if letras[j+1] == 'd' and letras[j+2] == 'l':
                        salida.append(silaba + letras[j+1])
                        silaba = ''
                        j += 2
                        continue
                    salida.append(silaba)
                    silaba = ''
                    j+=1
                    continue
                else:
                    if letras[j+2] == 's' and estructura[j+3] in 'LCO':
                        salida.append(silaba + letras[j+1] + letras[j+2])
                        silaba = ''
                        j += 3
                        continue
                    salida.append(silaba + letras[j+1])
                    silaba = ''
                    j += 2
                    continue
            else:
                if j+2 < len(letras):
                    if estructura[j+2] in 'AI':
                        salida.append(silaba)
                        silaba = ''
                        j+=1
                        continue
                    else:
                        if letras[j+2] == 's' and estructura[j+3] in 'LCO':
                            salida.append(silaba + letras[j+1] + letras[j+2])
                            silaba = ''
                            j += 3
                            continue
                        salida.append(silaba + letras[j+1])
                        silaba = ''
                        j += 2
                        continue
                else:
                    salida.append(silaba + letras[j+1])
                    silaba = ''
                    j += 2
                    continue
        elif estructura[j] == 'O':
            if estructura[j+1] in 'AIL':
                j+=1
                continue
            else:
                if letras[j+1] == '':
                    salida.append(silaba)
                    break
                raise TypeError("Estructura de sílaba incorrecta en la palabra {0}".format(word))
        else:
            if estructura[j+1] in 'AI':
                j+=1
                continue
            else:
                if letras[j+1] == '':
                    salida.append(silaba)
                    break
                elif letras[j+1] == 's':
                    salida.append(silaba)
                    silaba = ''
                    j+=1
                    continue
                raise TypeError("Estructura de sílaba incorrecta en la palabra {0}".format(word))
    return sep.join(salida)


Writing Silabas.py


# Preprocess Corpus

## Imports

In [0]:
import re
import operator
#from separadorSilabas import silabas # github
from Silabas import silabas # local

### atracción

In [7]:
word_atraccion = 'atracción'
silabas(word_atraccion)

TypeError: ignored

## add_space_file

In [0]:
def add_space_file(path_in, path_out):
    with open(path_out, 'w') as f1:
        with open(path_in) as f2:
            for line in f2:
                s = re.sub('([.,;:¿¡!?"()\´])', r' \1 ', line)
                s = re.sub('\s{2,}', ' ', s)
                f1.write(s+'\n')

In [0]:

path_in = './data/horoscopo_test_overfitting.txt'
path_out = './data/horoscopo_test_overfitting_spaces.txt'

add_space_file(path_in = path_in,
               path_out = path_out)

In [0]:
!ls data

In [0]:
#!rm -R ./data/horoscopo_test_overfitting_spaces.txt
#!ls data

In [0]:
!ls data

## get_syllables

In [0]:
def get_syllables(word, middle, end):
    '''Uses separadorSilabas to form a list of syllables
    Raises TypeError:
         if word is not separable i.e. if it's not a spanish word
    Args:
        word: string with the word to be separate
        middle: char to be added at the end of the middle syllables
        end: char to be added at the end of the end syllable
    Returns:
        List with syllables
    '''
    word_syllables = silabas(word).split('-')
    word_syllables = [word + middle for word in word_syllables]
    word_syllables[-1] = word_syllables[-1][0:-1] + end
    
    return word_syllables

In [0]:
word = 'palabra'

get_syllables(word = word,
              middle = '-',
              end = ':')

In [0]:
word = 'atracción'

get_syllables(word = word,
              middle = '-',
              end = ':')

## get_characters

In [0]:
def get_characters(syllable, middle = '-', end = ':'):
  
    letters = 'aáeéoóíúiuübcdfghjklmnñopqrstvwxyz'
    s = []
    
    for letter in syllable:
        if letter in letters:
            s += [letter + middle]

    if syllable[-1] == end:
        s[-1] = s[-1].replace(middle, end)
        
    return s

In [0]:
syllable = 'palabra'

get_characters(syllable = syllable,
               middle = '-',
               end = ':')

In [0]:
syllable = 'pa-'

get_characters(syllable = syllable,
               middle = '-',
               end = ':')

In [0]:
syllable = 'bra:'

get_characters(syllable = syllable,
               middle = '-',
               end = ':')

## punctuation, characters, to_ignore

In [0]:
# Punctuation

map_punctuation = {'¿': '<ai>', '?': '<ci>', '.': '<pt>', '\n': '<nl>', ',': '<cm>', '"':'<qt>'}
punctuation = set(map_punctuation)


# Characters

letters = 'aáeéoóíúiuübcdfghjklmnñopqrstvwxyz'
letters += letters.upper()
accepted_chars = set(letters)
characters = accepted_chars


# to_ignore

sign_to_ignore=[]
word_to_ignore=[]

to_ignore = [i for i in punctuation]
to_ignore = to_ignore + sign_to_ignore + word_to_ignore

## get_freq_words

In [0]:
def get_freq_words(word, freq_word, to_ignore = []):
    
    if word in to_ignore:
        return freq_word

    if word in freq_word:
        freq_word[word] += 1
    else:
        freq_word[word] = 1
    
    return freq_word

In [0]:
freq_word = dict()

word1 = 'hola'
word2 = 'chabelalaila'


In [0]:
get_freq_words(word = word1,
               freq_word = freq_word,
               to_ignore = to_ignore)

In [0]:
get_freq_words(word = word1,
               freq_word = freq_word,
               to_ignore = to_ignore)

In [0]:
get_freq_words(word = word2,
               freq_word = freq_word,
               to_ignore = to_ignore)

## word_to_syll

In [0]:
def word_to_syll(word, dict_word, to_ignore = [], middle='-', end=':'):
  
    if word in to_ignore:
        return dict_word

    if word not in dict_word:
        try:
            dict_word[word] = get_syllables(word, middle, end)
        except TypeError:
            print("word not considered for syllables: '{}'".format(word))
            
    return dict_word

In [0]:
dict_word = dict()

In [0]:
word_to_syll(word = word1,
             dict_word = dict_word,
             to_ignore = to_ignore)

In [0]:
word_to_syll(word = word2,
             dict_word = dict_word,
             to_ignore = to_ignore)

### atracción

In [0]:
word_atraccion = 'atracción'

dict_word_atraccion = dict()

word_to_syll(word = word_atraccion,
             dict_word = dict_word_atraccion,
             to_ignore = to_ignore)




## syll_to_char

In [0]:
def syll_to_charac(word, dict_syll, dict_word, to_ignore = [], middle='-', end=':'):
  
    if word in to_ignore:
        return dict_syll
      
    try:
        syllables = dict_word[word]
    except KeyError:
        print("word not exist in dictionary: '{}'".format(word))
        return dict_syll

    for syll in syllables:
        if syll not in dict_syll:
            dict_syll[syll] = get_characters(syll, middle, end)

    return dict_syll

In [0]:
dict_syll = dict()

In [0]:
syll_to_charac(word = word1,
               dict_syll = dict_syll,
               dict_word = dict_word,
               to_ignore = to_ignore)

In [0]:
syll_to_charac(word = word2,
               dict_syll = dict_syll,
               dict_word = dict_word,
               to_ignore = to_ignore)

## get_freq_syllables

In [0]:
def get_freq_syllables(freq_word, dict_word, to_ignore = []):
  
    freq_syll = dict()
    
    for k,v in freq_word.items():
        if k in to_ignore:
            pass

        try:
            syllables = dict_word[k]
        except KeyError:
            continue

        for s in syllables:
            if s in freq_syll:
                freq_syll[s] += v
            else:
                freq_syll[s] = v
    
    return freq_syll

In [0]:
get_freq_syllables(freq_word = freq_word,
                   dict_word = dict_word,
                   to_ignore = to_ignore)

## tokenize_corpus

In [0]:
def tokenize_corpus(path_file, to_ignore = []):
  
    dict_word = dict()
    dict_syll = dict()
    freq_word = dict()
    
    with open(path_file, 'r') as f1:
    
        for line in f1:
            words = line.lower().split()
            
            for w in words:
              
                dict_word = word_to_syll(w, dict_word, to_ignore)
                dict_syll = syll_to_charac(w, dict_syll, dict_word, to_ignore)
                freq_word = get_freq_words(w, freq_word, to_ignore)
                
    freq_syll = get_freq_syllables(freq_word, dict_word, to_ignore)

    return dict_word, dict_syll, freq_word, freq_syll

In [0]:
dict_word, dict_syll, freq_word, freq_syll = tokenize_corpus(path_file = path_out,
                                                             to_ignore = to_ignore)

In [0]:
dict_word

In [0]:
dict_syll

In [0]:
freq_word

In [0]:
freq_syll

## get_most_frequent

In [0]:
def get_most_frequent(freq_dict, quantity, to_ignore = []):
  
    '''Selects most frequent tokens.
    Args:
        freq_dict: list of tokens and frequent in corpus from where to select
        quantity: number that indicates the elements to be select from the list, if it's between [0,1] it's used as percentage
    Returns:
        set of length less than or equal to quantity (percentage*len(different tokens)) containing the most frequent tokens
    '''
    
    most_freq = set()
    max_tokens = int(len(freq_dict)*quantity) if quantity <= 1 else quantity
    #order = sorted(freq_dict.items(), key=freq_dict.get, reverse=True)
    
    for token, freq in sorted(freq_dict.items(), key=operator.itemgetter(1), reverse=True):
      
        if len(most_freq) < max_tokens:
           if token not in to_ignore:
                most_freq.add(token)
                
    return most_freq

In [0]:
quantity_words = 50

words = get_most_frequent(freq_dict = freq_word,
                          quantity = quantity_words,
                          to_ignore = to_ignore)


quantity_syll = 1000

syllables = get_most_frequent(freq_dict = freq_syll,
                              quantity = quantity_syll,
                              to_ignore = [w for w in words]) # Count syllables ignoring the most frequent words

In [0]:
words

In [0]:
syllables